In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 4)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 3.36992084980011, Final Batch Loss: 1.1454048156738281
Epoch 2, Loss: 3.3366039991378784, Final Batch Loss: 1.1298075914382935
Epoch 3, Loss: 3.292970299720764, Final Batch Loss: 1.1044026613235474
Epoch 4, Loss: 3.2573505640029907, Final Batch Loss: 1.0736380815505981
Epoch 5, Loss: 3.255733013153076, Final Batch Loss: 1.0895518064498901
Epoch 6, Loss: 3.2408639192581177, Final Batch Loss: 1.0802416801452637
Epoch 7, Loss: 3.2171924114227295, Final Batch Loss: 1.0656620264053345
Epoch 8, Loss: 3.2212233543395996, Final Batch Loss: 1.0794132947921753
Epoch 9, Loss: 3.179232954978943, Final Batch Loss: 1.0497808456420898
Epoch 10, Loss: 3.1799468994140625, Final Batch Loss: 1.063114881515503
Epoch 11, Loss: 3.177958369255066, Final Batch Loss: 1.0764403343200684
Epoch 12, Loss: 3.1267054080963135, Final Batch Loss: 1.0341405868530273
Epoch 13, Loss: 3.101980686187744, Final Batch Loss: 1.0312200784683228
Epoch 14, Loss: 3.0814080238342285, Final Batch Loss: 1.032398581504

Epoch 118, Loss: 0.3317769803106785, Final Batch Loss: 0.1893564611673355
Epoch 119, Loss: 0.22267486155033112, Final Batch Loss: 0.06096503138542175
Epoch 120, Loss: 0.3527325242757797, Final Batch Loss: 0.09014144539833069
Epoch 121, Loss: 0.2446000799536705, Final Batch Loss: 0.0927848294377327
Epoch 122, Loss: 0.4975327104330063, Final Batch Loss: 0.35922664403915405
Epoch 123, Loss: 0.34467407315969467, Final Batch Loss: 0.10626046359539032
Epoch 124, Loss: 0.24838127568364143, Final Batch Loss: 0.0543101541697979
Epoch 125, Loss: 0.23488404601812363, Final Batch Loss: 0.05662158876657486
Epoch 126, Loss: 0.38928162306547165, Final Batch Loss: 0.19245612621307373
Epoch 127, Loss: 0.18876244872808456, Final Batch Loss: 0.05229625850915909
Epoch 128, Loss: 0.19033879693597555, Final Batch Loss: 0.011469869874417782
Epoch 129, Loss: 0.2134115919470787, Final Batch Loss: 0.04569191485643387
Epoch 130, Loss: 0.19302432239055634, Final Batch Loss: 0.010876908898353577
Epoch 131, Loss: 0

Epoch 235, Loss: 0.35335410200059414, Final Batch Loss: 0.2611343562602997
Epoch 236, Loss: 0.2514330670237541, Final Batch Loss: 0.11579086631536484
Epoch 237, Loss: 0.33684225007891655, Final Batch Loss: 0.23563531041145325
Epoch 238, Loss: 0.12556118331849575, Final Batch Loss: 0.02112170122563839
Epoch 239, Loss: 0.21396151185035706, Final Batch Loss: 0.08606382459402084
Epoch 240, Loss: 0.12277889251708984, Final Batch Loss: 0.005810510367155075
Epoch 241, Loss: 0.1478027068078518, Final Batch Loss: 0.04326197877526283
Epoch 242, Loss: 0.1352172028273344, Final Batch Loss: 0.016716977581381798
Epoch 243, Loss: 0.12478489801287651, Final Batch Loss: 0.037265509366989136
Epoch 244, Loss: 0.14737479761242867, Final Batch Loss: 0.034180980175733566
Epoch 245, Loss: 0.1869364008307457, Final Batch Loss: 0.08727536350488663
Epoch 246, Loss: 0.12350593321025372, Final Batch Loss: 0.029864026233553886
Epoch 247, Loss: 0.11285614408552647, Final Batch Loss: 0.007017413154244423
Epoch 248, 

Epoch 351, Loss: 0.15280137583613396, Final Batch Loss: 0.07003853470087051
Epoch 352, Loss: 0.0944706650916487, Final Batch Loss: 0.0019958356861025095
Epoch 353, Loss: 0.12757786829024553, Final Batch Loss: 0.014830009080469608
Epoch 354, Loss: 0.24963472411036491, Final Batch Loss: 0.14696355164051056
Epoch 355, Loss: 0.1513015404343605, Final Batch Loss: 0.044184695929288864
Epoch 356, Loss: 0.11791790090501308, Final Batch Loss: 0.030942440032958984
Epoch 357, Loss: 0.0894550159573555, Final Batch Loss: 0.01679830253124237
Epoch 358, Loss: 0.06986518716439605, Final Batch Loss: 0.005267362575978041
Epoch 359, Loss: 0.13744144514203072, Final Batch Loss: 0.059520475566387177
Epoch 360, Loss: 0.2888565883040428, Final Batch Loss: 0.22814173996448517
Epoch 361, Loss: 0.1306306030601263, Final Batch Loss: 0.021023664623498917
Epoch 362, Loss: 0.07437248580390587, Final Batch Loss: 0.0006309492164291441
Epoch 363, Loss: 0.11347543820738792, Final Batch Loss: 0.012810468673706055
Epoch 

Epoch 468, Loss: 0.10108701745048165, Final Batch Loss: 0.002932580653578043
Epoch 469, Loss: 0.11402864940464497, Final Batch Loss: 0.028007442131638527
Epoch 470, Loss: 0.08178546465933323, Final Batch Loss: 0.01751861348748207
Epoch 471, Loss: 0.08843241259455681, Final Batch Loss: 0.019337279722094536
Epoch 472, Loss: 0.11709127947688103, Final Batch Loss: 0.06860532611608505
Epoch 473, Loss: 0.09719687141478062, Final Batch Loss: 0.018913017585873604
Epoch 474, Loss: 0.08286744356155396, Final Batch Loss: 0.027579110115766525
Epoch 475, Loss: 0.06130481942091137, Final Batch Loss: 0.0014423715183511376
Epoch 476, Loss: 0.05355798173695803, Final Batch Loss: 0.00318694394081831
Epoch 477, Loss: 0.06863703392446041, Final Batch Loss: 0.008746970444917679
Epoch 478, Loss: 0.0794019689783454, Final Batch Loss: 0.010382859967648983
Epoch 479, Loss: 0.1856946237385273, Final Batch Loss: 0.14203889667987823
Epoch 480, Loss: 0.06313694827258587, Final Batch Loss: 0.011094504967331886
Epoc

...

Epoch 4526, Loss: 0.004161923716310412, Final Batch Loss: 0.0024876301176846027
Epoch 4527, Loss: 0.12646779645001516, Final Batch Loss: 0.11984502524137497
Epoch 4528, Loss: 0.004891301796305925, Final Batch Loss: 0.0013840701431035995
Epoch 4529, Loss: 0.039240625221282244, Final Batch Loss: 0.011510021984577179
Epoch 4530, Loss: 0.034421845106408, Final Batch Loss: 0.0011228236835449934
Epoch 4531, Loss: 0.03379515504866504, Final Batch Loss: 5.301483270159224e-06
Epoch 4532, Loss: 0.0442059182387311, Final Batch Loss: 0.00017752012354321778
Epoch 4533, Loss: 0.04416179787222063, Final Batch Loss: 3.4821205190382898e-06
Epoch 4534, Loss: 0.017808279953897, Final Batch Loss: 9.973510168492794e-05
Epoch 4535, Loss: 0.10608196933753788, Final Batch Loss: 0.09101996570825577
Epoch 4536, Loss: 0.024100667953462107, Final Batch Loss: 2.9060331144137308e-05
Epoch 4537, Loss: 0.05919541139155626, Final Batch Loss: 0.035427581518888474
Epoch 4538, Loss: 0.009050880675204098, Final Batch Loss

Epoch 4638, Loss: 0.0005411033446307556, Final Batch Loss: 9.72492898654309e-07
Epoch 4639, Loss: 0.0004106286887690658, Final Batch Loss: 1.0025754818343557e-05
Epoch 4640, Loss: 0.004540960454050946, Final Batch Loss: 1.9575204532884527e-06
Epoch 4641, Loss: 0.0013784910770482384, Final Batch Loss: 8.157613774528727e-05
Epoch 4642, Loss: 0.0005146488224454515, Final Batch Loss: 4.31656053478946e-06
Epoch 4643, Loss: 0.001858105188148329, Final Batch Loss: 4.571548561216332e-05
Epoch 4644, Loss: 0.008214912289986387, Final Batch Loss: 0.0026001608930528164
Epoch 4645, Loss: 0.0005054292159911711, Final Batch Loss: 5.501459600054659e-05
Epoch 4646, Loss: 0.0005601103694061749, Final Batch Loss: 0.00014196039410308003
Epoch 4647, Loss: 0.00044260641880100593, Final Batch Loss: 6.685128028038889e-05
Epoch 4648, Loss: 0.015209081664579571, Final Batch Loss: 2.4413771825493313e-05
Epoch 4649, Loss: 0.00274837261531502, Final Batch Loss: 0.0009341362747363746
Epoch 4650, Loss: 0.00077353738

Epoch 4741, Loss: 0.00020556280651362613, Final Batch Loss: 4.5173401304055005e-06
Epoch 4742, Loss: 0.0011228289713471895, Final Batch Loss: 1.4566749086952768e-05
Epoch 4743, Loss: 0.01233058291109046, Final Batch Loss: 0.0041228593327105045
Epoch 4744, Loss: 0.0006123665771156084, Final Batch Loss: 1.3638968084705994e-05
Epoch 4745, Loss: 0.0009320639219367877, Final Batch Loss: 0.00019568846619222313
Epoch 4746, Loss: 0.0007699498382862657, Final Batch Loss: 4.285146133042872e-06
Epoch 4747, Loss: 0.0004407842097862158, Final Batch Loss: 3.3574997360119596e-05
Epoch 4748, Loss: 0.0001686388877715217, Final Batch Loss: 1.7157706679427065e-05
Epoch 4749, Loss: 0.0020020193260279484, Final Batch Loss: 0.0016144460532814264
Epoch 4750, Loss: 0.00037638067715306534, Final Batch Loss: 1.3765008588961791e-05
Epoch 4751, Loss: 0.0014628787098445173, Final Batch Loss: 2.484542619640706e-06
Epoch 4752, Loss: 0.00575296243550838, Final Batch Loss: 0.0004498915805015713
Epoch 4753, Loss: 0.000

Epoch 4841, Loss: 0.014917668784619309, Final Batch Loss: 2.7495218091644347e-05
Epoch 4842, Loss: 0.0022048330611141864, Final Batch Loss: 0.002015194622799754
Epoch 4843, Loss: 0.0005759301798207161, Final Batch Loss: 3.6012711461808067e-06
Epoch 4844, Loss: 0.00017507362281321548, Final Batch Loss: 5.210060044191778e-05
Epoch 4845, Loss: 0.008516021454852307, Final Batch Loss: 3.102296977885999e-05
Epoch 4846, Loss: 0.00029826812806277303, Final Batch Loss: 8.131023605528753e-06
Epoch 4847, Loss: 0.0005560231074923649, Final Batch Loss: 1.7528582247905433e-05
Epoch 4848, Loss: 0.00532592448143987, Final Batch Loss: 0.004708783235400915
Epoch 4849, Loss: 0.0004309985845338815, Final Batch Loss: 1.4932477370166453e-06
Epoch 4850, Loss: 0.00044418166726245545, Final Batch Loss: 0.00029757688753306866
Epoch 4851, Loss: 0.0004327388724050252, Final Batch Loss: 2.6961721232510172e-05
Epoch 4852, Loss: 0.0009526081812509801, Final Batch Loss: 1.543853613839019e-05
Epoch 4853, Loss: 0.00044

Epoch 4952, Loss: 0.000261132914602058, Final Batch Loss: 1.2296011846046895e-05
Epoch 4953, Loss: 0.0002564797432569321, Final Batch Loss: 1.8982766050612554e-05
Epoch 4954, Loss: 0.000733803171897307, Final Batch Loss: 3.639722126536071e-05
Epoch 4955, Loss: 0.0005146955636519124, Final Batch Loss: 7.591706889797933e-07
Epoch 4956, Loss: 0.0001334653679805342, Final Batch Loss: 1.4710978575749323e-05
Epoch 4957, Loss: 0.0001704275873635197, Final Batch Loss: 3.187256879755296e-06
Epoch 4958, Loss: 0.00044330384844215587, Final Batch Loss: 0.0002363045496167615
Epoch 4959, Loss: 9.396259702043608e-05, Final Batch Loss: 1.2622298527276143e-05
Epoch 4960, Loss: 0.0003714692161338462, Final Batch Loss: 4.8623101065459196e-06
Epoch 4961, Loss: 0.00018006292491179465, Final Batch Loss: 4.3919200720665685e-08
Epoch 4962, Loss: 0.00028427985864709626, Final Batch Loss: 1.3489426464730059e-06
Epoch 4963, Loss: 0.00016143254060807521, Final Batch Loss: 5.640234576276271e-06
Epoch 4964, Loss: 0

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[45  0  0]
 [ 0 47  0]
 [ 0  0 38]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        45
           1    1.00000   1.00000   1.00000        47
           2    1.00000   1.00000   1.00000        38

    accuracy                        1.00000       130
   macro avg    1.00000   1.00000   1.00000       130
weighted avg    1.00000   1.00000   1.00000       130



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 4)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 4)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 4) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[40  0  0]
 [ 1 39  0]
 [ 0  0 40]]
              precision    recall  f1-score   support

         0.0    0.97561   1.00000   0.98765        40
         1.0    1.00000   0.97500   0.98734        40
         2.0    1.00000   1.00000   1.00000        40

    accuracy                        0.99167       120
   macro avg    0.99187   0.99167   0.99167       120
weighted avg    0.99187   0.99167   0.99167       120



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    else:
        y[k] = 3

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.191941738128662, Final Batch Loss: 1.3773088455200195
Epoch 2, Loss: 4.216180443763733, Final Batch Loss: 1.4106996059417725
Epoch 3, Loss: 4.2418776750564575, Final Batch Loss: 1.4424982070922852
Epoch 4, Loss: 4.227945566177368, Final Batch Loss: 1.4305928945541382
Epoch 5, Loss: 4.261879563331604, Final Batch Loss: 1.472856879234314
Epoch 6, Loss: 4.2112215757369995, Final Batch Loss: 1.4219396114349365
Epoch 7, Loss: 4.200379848480225, Final Batch Loss: 1.4116337299346924
Epoch 8, Loss: 4.1935014724731445, Final Batch Loss: 1.404161810874939
Epoch 9, Loss: 4.190621495246887, Final Batch Loss: 1.402277946472168
Epoch 10, Loss: 4.184243679046631, Final Batch Loss: 1.4017337560653687
Epoch 11, Loss: 4.162977576255798, Final Batch Loss: 1.3807376623153687
Epoch 12, Loss: 4.190198063850403, Final Batch Loss: 1.4114370346069336
Epoch 13, Loss: 4.155712366104126, Final Batch Loss: 1.3763595819473267
Epoch 14, Loss: 4.154235482215881, Final Batch Loss: 1.3766350746154785
E

Epoch 116, Loss: 2.235073983669281, Final Batch Loss: 0.7390227317810059
Epoch 117, Loss: 2.2658209204673767, Final Batch Loss: 0.8816837668418884
Epoch 118, Loss: 2.2841984629631042, Final Batch Loss: 0.8734487295150757
Epoch 119, Loss: 2.1272794008255005, Final Batch Loss: 0.6979336738586426
Epoch 120, Loss: 2.177486836910248, Final Batch Loss: 0.7126415371894836
Epoch 121, Loss: 2.254896640777588, Final Batch Loss: 0.8607078194618225
Epoch 122, Loss: 2.0478065609931946, Final Batch Loss: 0.6463191509246826
Epoch 123, Loss: 2.2082031965255737, Final Batch Loss: 0.7810679078102112
Epoch 124, Loss: 1.8926373422145844, Final Batch Loss: 0.4493829309940338
Epoch 125, Loss: 2.1394415497779846, Final Batch Loss: 0.7836971282958984
Epoch 126, Loss: 2.168992817401886, Final Batch Loss: 0.7479113340377808
Epoch 127, Loss: 1.9315857887268066, Final Batch Loss: 0.5576208829879761
Epoch 128, Loss: 2.1324456334114075, Final Batch Loss: 0.7419420480728149
Epoch 129, Loss: 2.3989664912223816, Final

Epoch 238, Loss: 1.4513638317584991, Final Batch Loss: 0.4516041874885559
Epoch 239, Loss: 1.3807607591152191, Final Batch Loss: 0.3655708432197571
Epoch 240, Loss: 1.3609640896320343, Final Batch Loss: 0.3416200578212738
Epoch 241, Loss: 1.4447070360183716, Final Batch Loss: 0.49941501021385193
Epoch 242, Loss: 1.5801140367984772, Final Batch Loss: 0.573606014251709
Epoch 243, Loss: 1.6609793305397034, Final Batch Loss: 0.6722415685653687
Epoch 244, Loss: 1.493641585111618, Final Batch Loss: 0.4580710530281067
Epoch 245, Loss: 1.486601173877716, Final Batch Loss: 0.4639323949813843
Epoch 246, Loss: 1.525831937789917, Final Batch Loss: 0.5430471897125244
Epoch 247, Loss: 1.4745982587337494, Final Batch Loss: 0.5043842196464539
Epoch 248, Loss: 1.4863451719284058, Final Batch Loss: 0.45730137825012207
Epoch 249, Loss: 1.6040142178535461, Final Batch Loss: 0.6331064701080322
Epoch 250, Loss: 1.5612459778785706, Final Batch Loss: 0.5625686645507812
Epoch 251, Loss: 1.52831569314003, Final

Epoch 367, Loss: 0.9588837325572968, Final Batch Loss: 0.2224292755126953
Epoch 368, Loss: 1.2513839900493622, Final Batch Loss: 0.5081381797790527
Epoch 369, Loss: 1.1495431661605835, Final Batch Loss: 0.44022706151008606
Epoch 370, Loss: 1.1062564253807068, Final Batch Loss: 0.36789798736572266
Epoch 371, Loss: 0.9932963252067566, Final Batch Loss: 0.2732655107975006
Epoch 372, Loss: 1.0965668857097626, Final Batch Loss: 0.3051631450653076
Epoch 373, Loss: 1.1517189145088196, Final Batch Loss: 0.39363232254981995
Epoch 374, Loss: 0.8663430213928223, Final Batch Loss: 0.16970103979110718
Epoch 375, Loss: 1.2753191888332367, Final Batch Loss: 0.5631345510482788
Epoch 376, Loss: 1.0546168982982635, Final Batch Loss: 0.38109728693962097
Epoch 377, Loss: 1.0032325088977814, Final Batch Loss: 0.31977376341819763
Epoch 378, Loss: 1.2648912370204926, Final Batch Loss: 0.5651450157165527
Epoch 379, Loss: 1.1463593244552612, Final Batch Loss: 0.4172988831996918
Epoch 380, Loss: 1.0134764909744

Epoch 478, Loss: 1.1895917356014252, Final Batch Loss: 0.5328016877174377
Epoch 479, Loss: 1.2737302780151367, Final Batch Loss: 0.5707862377166748
Epoch 480, Loss: 1.0092034935951233, Final Batch Loss: 0.39815038442611694
Epoch 481, Loss: 1.055022269487381, Final Batch Loss: 0.3358890414237976
Epoch 482, Loss: 0.8046786934137344, Final Batch Loss: 0.16472573578357697
Epoch 483, Loss: 0.9100694954395294, Final Batch Loss: 0.3191549777984619
Epoch 484, Loss: 0.9914953112602234, Final Batch Loss: 0.3452851176261902
Epoch 485, Loss: 0.9163239598274231, Final Batch Loss: 0.21950548887252808
Epoch 486, Loss: 1.0605765879154205, Final Batch Loss: 0.4498912990093231
Epoch 487, Loss: 0.994173139333725, Final Batch Loss: 0.3868243098258972
Epoch 488, Loss: 1.2501992881298065, Final Batch Loss: 0.5639543533325195
Epoch 489, Loss: 0.9315674006938934, Final Batch Loss: 0.26841115951538086
Epoch 490, Loss: 1.169798195362091, Final Batch Loss: 0.49985161423683167
Epoch 491, Loss: 0.9925616085529327,

...

Epoch 4497, Loss: 0.34674860537052155, Final Batch Loss: 0.1911475658416748
Epoch 4498, Loss: 0.27531222254037857, Final Batch Loss: 0.08853881061077118
Epoch 4499, Loss: 0.27764661610126495, Final Batch Loss: 0.09710202366113663
Epoch 4500, Loss: 0.3398411348462105, Final Batch Loss: 0.07447347790002823
Epoch 4501, Loss: 0.3409449830651283, Final Batch Loss: 0.11737401783466339
Epoch 4502, Loss: 0.3113831803202629, Final Batch Loss: 0.09695363789796829
Epoch 4503, Loss: 0.20599961280822754, Final Batch Loss: 0.05331127345561981
Epoch 4504, Loss: 0.19820035807788372, Final Batch Loss: 0.023701762780547142
Epoch 4505, Loss: 0.20976159535348415, Final Batch Loss: 0.007130226120352745
Epoch 4506, Loss: 0.5275391265749931, Final Batch Loss: 0.35749807953834534
Epoch 4507, Loss: 0.16113269375637174, Final Batch Loss: 0.006009094882756472
Epoch 4508, Loss: 0.2582598403096199, Final Batch Loss: 0.0971977487206459
Epoch 4509, Loss: 0.1859649159014225, Final Batch Loss: 0.06485792249441147
Epoc

Epoch 4603, Loss: 0.2046114206314087, Final Batch Loss: 0.10029895603656769
Epoch 4604, Loss: 0.2061837688088417, Final Batch Loss: 0.0532427653670311
Epoch 4605, Loss: 0.1580549692735076, Final Batch Loss: 0.013713008724153042
Epoch 4606, Loss: 0.21271383576095104, Final Batch Loss: 0.024988515302538872
Epoch 4607, Loss: 0.352896086871624, Final Batch Loss: 0.1393059492111206
Epoch 4608, Loss: 0.20147307962179184, Final Batch Loss: 0.03901170939207077
Epoch 4609, Loss: 0.2607291229069233, Final Batch Loss: 0.1350591629743576
Epoch 4610, Loss: 0.19457115232944489, Final Batch Loss: 0.041713275015354156
Epoch 4611, Loss: 0.19032463058829308, Final Batch Loss: 0.028936166316270828
Epoch 4612, Loss: 0.22893776558339596, Final Batch Loss: 0.028951002284884453
Epoch 4613, Loss: 0.17952912487089634, Final Batch Loss: 0.014436105266213417
Epoch 4614, Loss: 0.11241372860968113, Final Batch Loss: 0.0083878543227911
Epoch 4615, Loss: 0.18121474981307983, Final Batch Loss: 0.026370123028755188
Ep

Epoch 4711, Loss: 0.22850243002176285, Final Batch Loss: 0.054677724838256836
Epoch 4712, Loss: 0.20980833284556866, Final Batch Loss: 0.03117918036878109
Epoch 4713, Loss: 0.2060791626572609, Final Batch Loss: 0.055666081607341766
Epoch 4714, Loss: 0.20690560713410378, Final Batch Loss: 0.08408927172422409
Epoch 4715, Loss: 0.20067698508501053, Final Batch Loss: 0.06158033758401871
Epoch 4716, Loss: 0.17954575642943382, Final Batch Loss: 0.0323486402630806
Epoch 4717, Loss: 0.25687792524695396, Final Batch Loss: 0.13982908427715302
Epoch 4718, Loss: 0.25183432549238205, Final Batch Loss: 0.024748757481575012
Epoch 4719, Loss: 0.13307231501676142, Final Batch Loss: 0.0037814138922840357
Epoch 4720, Loss: 0.21826406195759773, Final Batch Loss: 0.03668062388896942
Epoch 4721, Loss: 0.22134467214345932, Final Batch Loss: 0.07599752396345139
Epoch 4722, Loss: 0.23729288205504417, Final Batch Loss: 0.04341668263077736
Epoch 4723, Loss: 0.2395290918648243, Final Batch Loss: 0.115710936486721

Epoch 4826, Loss: 0.16113224811851978, Final Batch Loss: 0.007798174396157265
Epoch 4827, Loss: 0.14657284133136272, Final Batch Loss: 0.028911082074046135
Epoch 4828, Loss: 0.3173672705888748, Final Batch Loss: 0.21511931717395782
Epoch 4829, Loss: 0.09804652677848935, Final Batch Loss: 0.007112570572644472
Epoch 4830, Loss: 0.21278127282857895, Final Batch Loss: 0.04140201210975647
Epoch 4831, Loss: 0.20740048587322235, Final Batch Loss: 0.04923931509256363
Epoch 4832, Loss: 0.15690824203193188, Final Batch Loss: 0.02600417099893093
Epoch 4833, Loss: 0.23799901455640793, Final Batch Loss: 0.09810525178909302
Epoch 4834, Loss: 0.2557941563427448, Final Batch Loss: 0.05039142444729805
Epoch 4835, Loss: 0.2699071578681469, Final Batch Loss: 0.11557645350694656
Epoch 4836, Loss: 0.16277288179844618, Final Batch Loss: 0.012621189467608929
Epoch 4837, Loss: 0.14877342758700252, Final Batch Loss: 0.004794541280716658
Epoch 4838, Loss: 0.1733928695321083, Final Batch Loss: 0.0172102302312850

Epoch 4940, Loss: 0.1431985516101122, Final Batch Loss: 0.005704609677195549
Epoch 4941, Loss: 0.13680581375956535, Final Batch Loss: 0.017534393817186356
Epoch 4942, Loss: 0.14226146787405014, Final Batch Loss: 0.05043976381421089
Epoch 4943, Loss: 0.17367278039455414, Final Batch Loss: 0.09450225532054901
Epoch 4944, Loss: 0.29535041749477386, Final Batch Loss: 0.1608518809080124
Epoch 4945, Loss: 0.2052983120083809, Final Batch Loss: 0.05724961683154106
Epoch 4946, Loss: 0.2078400959726423, Final Batch Loss: 0.00294278166256845
Epoch 4947, Loss: 0.22843338549137115, Final Batch Loss: 0.03595364838838577
Epoch 4948, Loss: 0.28562432900071144, Final Batch Loss: 0.1331131011247635
Epoch 4949, Loss: 0.17857392132282257, Final Batch Loss: 0.04738973453640938
Epoch 4950, Loss: 0.23791025951504707, Final Batch Loss: 0.04244257137179375
Epoch 4951, Loss: 0.18581206258386374, Final Batch Loss: 0.010339665226638317
Epoch 4952, Loss: 0.2134331464767456, Final Batch Loss: 0.06001193821430206
Ep

In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[42  0  0  0]
 [ 0 28  2  0]
 [ 0  0 25  2]
 [ 0  1  0 30]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        42
           1    0.96552   0.93333   0.94915        30
           2    0.92593   0.92593   0.92593        27
           3    0.93750   0.96774   0.95238        31

    accuracy                        0.96154       130
   macro avg    0.95724   0.95675   0.95686       130
weighted avg    0.96175   0.96154   0.96153       130



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[20  5  4  1]
 [ 1 24  3  2]
 [ 0  2 21  7]
 [ 0 10  0 20]]
              precision    recall  f1-score   support

           0    0.95238   0.66667   0.78431        30
           1    0.58537   0.80000   0.67606        30
           2    0.75000   0.70000   0.72414        30
           3    0.66667   0.66667   0.66667        30

    accuracy                        0.70833       120
   macro avg    0.73860   0.70833   0.71279       120
weighted avg    0.73860   0.70833   0.71279       120

